In [ ]:
!pip install wikipedia --quiet
!pip install transformers --quiet
#!pip install bert-for-tf2 --quiet
#!pip install tensorflow-datasets --quiet
#!python -m spacy download xx_ent_wiki_sm --quiet

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
languages = ['af',
'sq',
'am',
'ar',
'hy',
'as',
'az',
'eu',
'be',
'bn',
#'Bengali Romanize',
'bs',
'br',
'bg',
'my',
#'Burmese zawgyi font',
'ca',
'zh',
'zh-tw',
'hr',
'cs',
'da',
'nl',
'en',
'eo',
'et',
'tl',#'Filipino',
'fi',
'fr',
'gl',
'ka',
'de',
'el',
'gu',
'ha',
'he',
'hi',
#'Hindi Romanize',
'hu',
'is',
'id',
'ga',
'it',
'ja',
'jv',
'kn',
'kk',
'km',
'ko',
'ku',
#'Kyrgyz',
'lo',
'la',
'lv',
'lt',
'mk',
'mg',
'ms',
'ml',
'mr',
'mn',
'ne',
'no',
'or',
'om',
'ps',
'fa',
'pl',
'pt',
'pa',
'ro',
'ru',
'sa',
'gd',
'sr',
'sd',
#'Sinhala',
'sk',
'sl',
'so',
'es',
'su',
'sw',
'sv',
'ta',
#'Tamil Romanize',
'te',
#'Telugu Romanize',
'th',
'tr',
'uk',
'ur',
#'Urdu Romanize',
'ug',
'uz',
'vi',
'cy',
'fy',
'xh',
'yi']
import wikipedia
print(str(len(languages)) + ' languages')
for lan in languages:
    try:
        wikipedia.set_lang(lan)
    except:
        print(lan +' not working')
    


wikipedia.set_lang('en') # tcy not in multilingual bert
page = None
while(page == None):
    try:
        page = wikipedia.page(title = wikipedia.random())
    except:
        pass
line = [line for line in page.content.splitlines() if len(line.split()) > 0 and line.split()[0] !='==' and line.split()[0] !='===']
for l in line:
    print(l)
    #token = tokenizer.tokenize(l)
    #print(token)
    #print(tokenizer.convert_tokens_to_ids(token))

from bert import bert_tokenization
import spacy
import tensorflow as tf
tf.gfile = tf.io.gfile
tokenizer = bert_tokenization.FullTokenizer(do_lower_case=False ,vocab_file = '/kaggle/input/bertbase-multilingual-cased-new-recommended/multi_cased_L-12_H-768_A-12/vocab.txt')

nlp = spacy.load("xx_ent_wiki_sm")
doc =  nlp('貓，通常指家貓，為小型貓科動物。根據遺傳學及考古學分析，人類養貓的紀錄可追溯至10,000年前的新月沃土地區，古埃及人飼養貓的紀錄可追溯至3,600年前，目的可能為捕鼠及其他齧齒目動物，以防止牠們吃掉榖物。')
for sent in doc.sents:
    print(sent.text)
    token = tokenizer.tokenize(sent.text)
    print(token)
    print(tokenizer.convert_tokens_to_ids(token))

In [ ]:
import wikipedia
import tensorflow as tf
import random
from transformers import *
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
class WikiDataset(tf.data.Dataset):
    def _generator():
        for i in range(len(languages)):
            wikipedia.set_lang(languages[i])
            page = None
            line = None
            while(page == None):
                try:
                    page = wikipedia.page(title = wikipedia.random())
                    line = [line for line in page.content.splitlines() if len(line.split()) > 0 and line.split()[0] !='==' and line.split()[0] !='===']
                    if len(line) < 1:
                        page = None
                except:
                    pass
            
            line = random.choice(line)
            token = tokenizer.encode(line)
            yield {'token': token , 'y_token': tf.one_hot(token,tokenizer.vocab_size),'y_language': tf.one_hot(i,len(languages))}
    
    def __new__(cls):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types={'token': tf.int32,
                          'y_token': tf.float32,
                          'y_language': tf.float32},
            
            output_shapes={'token':tf.TensorShape([ None]),
                           'y_token': tf.TensorShape([None,tokenizer.vocab_size]),
                           'y_language': tf.TensorShape([len(languages)])}

        )
    def padding(token):
        WikiDataset.pad_id
        
        
def train_input_fn(epoch = 50):
    def dataset_generator_fun(*args):
        return WikiDataset()
    
    return (tf.data.Dataset.range(1)
            .interleave(dataset_generator_fun
                        , num_parallel_calls=tf.data.experimental.AUTOTUNE).repeat(epoch)#.shuffle(30)
            .prefetch(tf.data.experimental.AUTOTUNE)
           )

In [ ]:
import tensorflow as tf
from transformers import *
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')
model = TFXLMRobertaModel.from_pretrained('xlm-roberta-large', from_pt=True)
model.compile()

In [ ]:
ds = train_input_fn().enumerate()
i = 0
for line in ds:
    data = line
    result = model.predict(line[1]['token'])
    i = i+1
    if i > 0:
        break

In [ ]:
print(data[1]['token'])
print(data[1]['y_token'])

In [ ]:
result[0].shape